In [ ]:
# Specify hyperparameters
num_epochs = 50
learning_rate = 0.004
momentum = 0.9
batch_size = 100
num_workers = 6

# output labels (COVID or NON-COVID)
num_classes = 2

# Specify model to train
model_name = "InceptionV3"
assert model_name in ["ResNet18", "ResNet50", "DenseNet121", "SqueezeNet1.1", "InceptionV3", "COVIDNet", "COVIDNetLSTM"]

#from IPython.display import clear_output # This will clear previous print statements
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #This is to keep all print statements

In [ ]:
import torch.nn as nn
import torchvision
from datetime import datetime
from pathlib import Path
from covidnet import CovidNet
from covidnetlstm import CovidNetLSTM

def no_grad(model):
    for param in model.parameters():
        param.requires_grad = False
        
    return model
now = datetime.now()
now = now.strftime("%Y-%m-%d-%H%M")
Path(f'./TrainingResults/{model_name}/{now}').mkdir(parents=True, exist_ok=True)
f = open(f'./TrainingResults/{model_name}/{now}/hyperparameters.txt', 'w+')
f.write(f'epochs={num_epochs}\nlearning rate={learning_rate}\nbatch size={batch_size}')
f.close()

if model_name == "ResNet18":
    trained_model_path = f'TrainingResults/{model_name}/{now}/resnet18_pretrained.pt'
    model_conv = no_grad(torchvision.models.resnet18(pretrained=True))
    
    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, num_classes)
    
elif model_name == "ResNet50":
    trained_model_path = f'TrainingResults/{model_name}/{now}/resnet50_pretrained.pt'
    model_conv = no_grad(torchvision.models.resnet50(pretrained=True))
    
    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, num_classes)
         
elif model_name == "DenseNet121":
    trained_model_path = f'TrainingResults/{model_name}/{now}/densenet121_pretrained.pt'
    model_conv = no_grad(torchvision.models.densenet121(pretrained=True))
    
    num_ftrs = model_conv.classifier.in_features
    model_conv.classifier = nn.Linear(num_ftrs, num_classes)
    
elif model_name == "SqueezeNet1.1":
    trained_model_path = f'TrainingResults/{model_name}/{now}/squeezenet1-1_pretrained.pt'
    model_conv = no_grad(torchvision.models.squeezenet1_1(pretrained=True))
    
    # SqueezeNet fine tuning adapted from https://discuss.pytorch.org/t/fine-tuning-squeezenet/3855
    num_ftrs = 512
    model_conv.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Conv2d(num_ftrs, num_classes, kernel_size=1),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(13)
    )
    
elif model_name == "InceptionV3":
    trained_model_path = f'TrainingResults/{model_name}/{now}/inceptionv3_pretrained.pt'
    model_conv = no_grad(torchvision.models.inception_v3(pretrained=True))
    
    num_ftrs = model_conv.AuxLogits.fc.in_features
    model_conv.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)  
    num_ftrs = model_conv.fc.in_features        
    model_conv.fc = nn.Linear(num_ftrs, num_classes)

elif model_name == "COVIDNet":
    # CovidNet source code: https://github.com/iliasprc/COVIDNet/blob/master/model/model.py
    trained_model_path = f'TrainingResults/{model_name}/{now}/covidnet_trained.pt'
    model_conv = CovidNet(model='small', n_classes=num_classes)

elif model_name == "COVIDNetLSTM":
    # CovidNet source code: https://github.com/iliasprc/COVIDNet/blob/master/model/model.py
    # LSTM Inspiration: https://www.sciencedirect.com/science/article/pii/S2352914820305621
    trained_model_path = f'TrainingResults/{model_name}/{now}/covidnetlstm_trained.pt'
    model_conv = CovidNetLSTM(model='small', n_classes=num_classes)
    

In [ ]:
# Data transformations and subsequent training adapted from https://github.com/shervinmin/DeepCovid
from torchvision import transforms

img_size = 299 if (model_name == "InceptionV3") else 224

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(img_size),
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(img_size),
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

data_dir = './data/'

image_datasets = {x: ImageFolder(os.path.join(data_dir, x),
                                 data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=num_workers)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes  ## 0: child, and 1: nonchild

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
import copy
import time
import torch
from matplotlib import pyplot as plt
from termcolor import colored

%matplotlib

def train_model(name, model, criterion, optimizer, scheduler, trained_model_path):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    val_losses = []
    train_losses = []
    val_accuracy = []
    train_accuracy = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for cur_batch_ind, (inputs, labels) in enumerate(dataloaders[phase], start=1):
                batch_start_time = time.time()

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train' and name == 'InceptionV3':
                        outputs, aux_outputs = model(inputs)

                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                _, preds = torch.max(outputs, 1)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                matches_bool = (preds == labels.data).cpu()
                matches_sum = matches_bool.sum()
                running_corrects += matches_sum

            epoch_loss= running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracy.append(epoch_acc)
            else:
                val_losses.append(epoch_loss)
                val_accuracy.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model, trained_model_path)

        # Plot learning curve    
        plt.cla()
        plt.plot(range(len(val_losses)), val_losses, label='val')
        plt.plot(range(len(train_losses)), train_losses, label='train')
        plt.legend()
        plt.ylim(0,1)
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.suptitle("Training Losses/Epoch")
        plt.savefig(f'./TrainingResults/{model_name}/{now}/train-val-loss-by-epoch.png')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc= %.3f at Epoch: %d' %(best_acc,best_epoch) )
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
import time
from torch import optim
from torch.optim import lr_scheduler
from torchvision import models

start_time = time.time()

model_conv = model_conv.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as opposed to before
if (model_name == "ResNet18") or (model_name == "ResNet50"):
    optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=learning_rate, momentum=momentum)
elif (model_name == "DenseNet121") or (model_name == "SqueezeNet1.1"):
    optimizer_conv = optim.SGD(model_conv.classifier.parameters(), lr=learning_rate, momentum=momentum)
elif model_name == "InceptionV3":
    params_to_update = [p for _, p in model_conv.named_parameters() if p.requires_grad]
    optimizer_conv = optim.SGD(params_to_update, lr=learning_rate, momentum=momentum)
elif (model_name == "COVIDNet") or (model_name == "COVIDNetLSTM"):
    optimizer_conv = optim.SGD(model_conv.parameters(), lr=learning_rate, momentum=momentum)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_name, model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, trained_model_path)
model_conv.eval()
torch.save(model_conv, trained_model_path)

end_time= time.time()
print("total_time transfer learning=", end_time - start_time)
